# RSI Divergence

In [1]:
import pandas as pd
df = pd.read_csv("EURUSD_Candlestick_4_Hour_ASK_05.05.2003-16.10.2021.csv")
df.columns=['time', 'open', 'high', 'low', 'close', 'volume']
#Check if NA values are in data
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.head(10)

FileNotFoundError: [Errno 2] No such file or directory: 'EURUSD_Candlestick_4_Hour_ASK_05.05.2003-16.10.2021.csv'

In [ ]:
import numpy as np
import pandas_ta as ta
#df.ta.indicators()
#help(ta.rsi)
df['RSI'] = df.ta.rsi(length=14)

def myRSI(price, n=20):
    delta = price['close'].diff()
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    RolUp = dUp.rolling(window=n).mean()
    RolDown = dDown.rolling(window=n).mean().abs()
    
    RS = RolUp / RolDown
    rsi= 100.0 - (100.0 / (1.0 + RS))
    return rsi

#df['RSI'] = myRSI(df)

#df.dropna(inplace=True)
#df.reset_index(drop=True, inplace=True)
df.head(20)

In [ ]:
dfpl = df[28100:28200]
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

fig = make_subplots(rows=2, cols=1)
fig.append_trace(go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close']), row=1, col=1)
fig.append_trace(go.Scatter(
    x=dfpl.index,
    y=dfpl['RSI'],
), row=2, col=1)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
def pivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.low[l]>df1.low[i]):
            pividlow=0
        if(df1.high[l]<df1.high[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0

def RSIpivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0

    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.RSI[l]>df1.RSI[i]):
            pividlow=0
        if(df1.RSI[l]<df1.RSI[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0 


#pivotid(df,28145,5,5)

In [ ]:
df['pivot'] = df.apply(lambda x: pivotid(df, x.name,5,5), axis=1)
df['RSIpivot'] = df.apply(lambda x: RSIpivotid(df, x.name, 5, 5), axis=1)

In [ ]:
def pointpos(x):
    if x['pivot']==1:
        return x['low']-1e-3
    elif x['pivot']==2:
        return x['high']+1e-3
    else:
        return np.nan

def RSIpointpos(x):
    if x['RSIpivot']==1:
        return x['RSI']-1
    elif x['RSIpivot']==2:
        return x['RSI']+1
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)
df['RSIpointpos'] = df.apply(lambda row: RSIpointpos(row), axis=1)
#df[df.RSIpivot==1].count()

In [ ]:
dfpl = df[28000:28500]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
dfpl = df[28000:28150]
fig = make_subplots(rows=2, cols=1)
fig.append_trace(go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close']), row=1, col=1)

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=4, color="MediumPurple"),
                name="pivot", row=1, col=1)

fig.append_trace(go.Scatter(x=dfpl.index, y=dfpl['RSI']), row=2, col=1)
fig.add_scatter(x=dfpl.index, y=dfpl['RSIpointpos'], mode="markers",
                marker=dict(size=4, color="MediumPurple"),
                name="pivot", row=2, col=1)


fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
df

In [ ]:
import numpy as np
from matplotlib import pyplot
backcandles= 60

#candleid = 8800
candleid = 85

maxim = np.array([])
minim = np.array([])
xxmin = np.array([])
xxmax = np.array([])

maximRSI = np.array([])
minimRSI = np.array([])
xxminRSI = np.array([])
xxmaxRSI = np.array([])

for i in range(candleid-backcandles, candleid+1):
    if df.iloc[i].pivot == 1:
        minim = np.append(minim, df.iloc[i].low)
        xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
    if df.iloc[i].pivot == 2:
        maxim = np.append(maxim, df.iloc[i].high)
        xxmax = np.append(xxmax, i) # df.iloc[i].name
    if df.iloc[i].RSIpivot == 1:
        minimRSI = np.append(minimRSI, df.iloc[i].RSI)
        xxminRSI = np.append(xxminRSI, df.iloc[i].name)
    if df.iloc[i].RSIpivot == 2:
        maximRSI = np.append(maximRSI, df.iloc[i].RSI)
        xxmaxRSI = np.append(xxmaxRSI, df.iloc[i].name)
        
slmin, intercmin = np.polyfit(xxmin, minim,1)
slmax, intercmax = np.polyfit(xxmax, maxim,1)
slminRSI, intercminRSI = np.polyfit(xxminRSI, minimRSI,1)
slmaxRSI, intercmaxRSI = np.polyfit(xxmaxRSI, maximRSI,1)

print(slmin, slmax, slminRSI, slmaxRSI)


dfpl = df[candleid-backcandles-5:candleid+backcandles]
fig = make_subplots(rows=2, cols=1)
fig.append_trace(go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close']), row=1, col=1)
fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=4, color="MediumPurple"),
                name="pivot", row=1, col=1)
fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + intercmin, mode='lines', name='min slope'), row=1, col=1)
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + intercmax, mode='lines', name='max slope'), row=1, col=1)

fig.append_trace(go.Scatter(x=dfpl.index, y=dfpl['RSI']), row=2, col=1)
fig.add_scatter(x=dfpl.index, y=dfpl['RSIpointpos'], mode="markers",
                marker=dict(size=2, color="MediumPurple"),
                name="pivot", row=2, col=1)
fig.add_trace(go.Scatter(x=xxminRSI, y=slminRSI*xxminRSI + intercminRSI, mode='lines', name='min slope'), row=2, col=1)
fig.add_trace(go.Scatter(x=xxmaxRSI, y=slmaxRSI*xxmaxRSI + intercmaxRSI, mode='lines', name='max slope'), row=2, col=1)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

# Fitting pivots into slopes

In [ ]:
dfpl = df[0:500]
def divsignal(x, nbackcandles):
    backcandles=nbackcandles 
    candleid = int(x.name)

    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])

    maximRSI = np.array([])
    minimRSI = np.array([])
    xxminRSI = np.array([])
    xxmaxRSI = np.array([])

    for i in range(candleid-backcandles, candleid+1):
        if df.iloc[i].pivot == 1:
            minim = np.append(minim, df.iloc[i].low)
            xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
        if df.iloc[i].pivot == 2:
            maxim = np.append(maxim, df.iloc[i].high)
            xxmax = np.append(xxmax, i) # df.iloc[i].name
        if df.iloc[i].RSIpivot == 1:
            minimRSI = np.append(minimRSI, df.iloc[i].RSI)
            xxminRSI = np.append(xxminRSI, df.iloc[i].name)
        if df.iloc[i].RSIpivot == 2:
            maximRSI = np.append(maximRSI, df.iloc[i].RSI)
            xxmaxRSI = np.append(xxmaxRSI, df.iloc[i].name)

    if maxim.size<2 or minim.size<2 or maximRSI.size<2 or minimRSI.size<2:
        return 0
    
    slmin, intercmin = np.polyfit(xxmin, minim,1)
    slmax, intercmax = np.polyfit(xxmax, maxim,1)
    slminRSI, intercminRSI = np.polyfit(xxminRSI, minimRSI,1)
    slmaxRSI, intercmaxRSI = np.polyfit(xxmaxRSI, maximRSI,1)
    
    
    if slmin > 1e-4 and slmax > 1e-4 and slmaxRSI <-0.1:
        return 1
    elif slmin < -1e-4 and slmax < -1e-4 and slminRSI > 0.1:
        return 2
    else:
        return 0

dfpl['divSignal'] = dfpl.apply(lambda row: divsignal(row,30), axis=1)
dfpl

In [ ]:
#dfpl[dfpl.divSignal==1].count()
dfpl[dfpl.divSignal==1]

# pivot points levels instead of slopes

In [ ]:
dfpl = df[0:500]
def divsignal2(x, nbackcandles):
    backcandles=nbackcandles 
    candleid = int(x.name)

    closp = np.array([])
    xxclos = np.array([])
    
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])

    maximRSI = np.array([])
    minimRSI = np.array([])
    xxminRSI = np.array([])
    xxmaxRSI = np.array([])

    for i in range(candleid-backcandles, candleid+1):
        closp = np.append(closp, df.iloc[i].close)
        xxclos = np.append(xxclos, i)
        if df.iloc[i].pivot == 1:
            minim = np.append(minim, df.iloc[i].low)
            xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
        if df.iloc[i].pivot == 2:
            maxim = np.append(maxim, df.iloc[i].high)
            xxmax = np.append(xxmax, i) # df.iloc[i].name
        if df.iloc[i].RSIpivot == 1:
            minimRSI = np.append(minimRSI, df.iloc[i].RSI)
            xxminRSI = np.append(xxminRSI, df.iloc[i].name)
        if df.iloc[i].RSIpivot == 2:
            maximRSI = np.append(maximRSI, df.iloc[i].RSI)
            xxmaxRSI = np.append(xxmaxRSI, df.iloc[i].name)

    slclos, interclos = np.polyfit(xxclos, closp,1)
    
    if slclos > 1e-4 and (maximRSI.size<2 or maxim.size<2):
        return 0
    if slclos < -1e-4 and (minimRSI.size<2 or minim.size<2):
        return 0
# signal decisions here !!!
    if slclos > 1e-4:
        if maximRSI[-1]<maximRSI[-2] and maxim[-1]>maxim[-2]:
            return 1
    elif slclos < -1e-4:
        if minimRSI[-1]>minimRSI[-2] and minim[-1]<minim[-2]:
            return 2
    else:
        return 0

dfpl['divSignal2'] = dfpl.apply(lambda row: divsignal2(row,30), axis=1)
dfpl

In [ ]:
#dfpl[dfpl.divSignal2==2].count()
dfpl[dfpl.divSignal2==1]